# 07_cycle_analyzer.ipynb - Breathing Cycle Analysis

This notebook analyzes the 4D breathing cycle data to understand:
1. Data structure and consistency
2. Motion patterns between phases
3. Cycle timing and characteristics
4. Quality assessment for interpolation

**Output**: Analysis results for implementing frame interpolation

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import cv2
from scipy import ndimage
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuration
CYCLES_ROOT = Path("/mnt/tcia_data/processed/4D-Lung-Cycles/")
OUTPUT_DIR = Path("/mnt/tcia_data/processed/4D-Lung-Cycles/analysis")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Cycles root: {CYCLES_ROOT}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Cycles root exists: {CYCLES_ROOT.exists()}")

Cycles root: /mnt/tcia_data/processed/4D-Lung-Cycles
Output directory: /mnt/tcia_data/processed/4D-Lung-Cycles/analysis
Cycles root exists: True


## 1. Data Structure Analysis

In [2]:
# Scan directory structure
def scan_data_structure():
    """Scan and analyze the data structure"""
    
    structure_info = []
    
    if not CYCLES_ROOT.exists():
        print(f"ERROR: {CYCLES_ROOT} does not exist")
        return None
    
    # Get all patient directories
    patient_dirs = [d for d in CYCLES_ROOT.iterdir() if d.is_dir()]
    print(f"Found {len(patient_dirs)} patient directories")
    
    for patient_dir in tqdm(patient_dirs, desc="Scanning patients"):
        patient_id = patient_dir.name
        
        # Get all cycle directories for this patient
        cycle_dirs = [d for d in patient_dir.iterdir() if d.is_dir()]
        
        for cycle_dir in cycle_dirs:
            cycle_id = cycle_dir.name
            
            # Get all phase files
            phase_files = sorted(cycle_dir.glob("phase_*.npy"))
            
            if not phase_files:
                continue
                
            # Load first phase to get shape info
            try:
                first_phase = np.load(phase_files[0])
                
                structure_info.append({
                    'patient_id': patient_id,
                    'cycle_id': cycle_id,
                    'num_phases': len(phase_files),
                    'volume_shape': first_phase.shape,
                    'dtype': str(first_phase.dtype),
                    'file_size_mb': sum(f.stat().st_size for f in phase_files) / (1024**2)
                })
                
            except Exception as e:
                print(f"Error loading {patient_id}/{cycle_id}: {e}")
    
    return pd.DataFrame(structure_info)

# Run structure analysis
structure_df = scan_data_structure()
if structure_df is not None:
    print(f"\nStructure analysis complete: {len(structure_df)} cycles found")
    print(structure_df.head())
else:
    print("Structure analysis failed")

Found 23 patient directories


Scanning patients: 100%|██████████| 23/23 [00:07<00:00,  3.13it/s]


Structure analysis complete: 587 cycles found
    patient_id          cycle_id  num_phases     volume_shape   dtype  \
0  116_HM10395  116_HM10395_S106          10   (50, 512, 512)   int16   
1  116_HM10395  116_HM10395_S107          10   (50, 512, 512)   int16   
2  116_HM10395  116_HM10395_S301          10  (118, 512, 512)  uint16   
3  116_HM10395  116_HM10395_S118          10   (50, 512, 512)   int16   
4  116_HM10395  116_HM10395_S102          10   (50, 512, 512)   int16   

   file_size_mb  
0    250.001221  
1    250.001221  
2    590.001221  
3    250.001221  
4    250.001221  


In [3]:
# Basic statistics
if structure_df is not None:
    print("=== DATA STRUCTURE SUMMARY ===")
    print(f"Total cycles: {len(structure_df)}")
    print(f"Unique patients: {structure_df['patient_id'].nunique()}")
    print(f"Cycles per patient: {len(structure_df) / structure_df['patient_id'].nunique():.1f}")
    
    print("\n=== PHASE COUNT DISTRIBUTION ===")
    phase_counts = structure_df['num_phases'].value_counts().sort_index()
    for phases, count in phase_counts.items():
        print(f"{phases} phases: {count} cycles ({count/len(structure_df)*100:.1f}%)")
    
    print("\n=== VOLUME SHAPES ===")
    shape_counts = structure_df['volume_shape'].value_counts()
    for shape, count in shape_counts.items():
        print(f"{shape}: {count} cycles ({count/len(structure_df)*100:.1f}%)")
    
    print("\n=== FILE SIZE STATISTICS ===")
    print(f"Total data size: {structure_df['file_size_mb'].sum():.1f} MB")
    print(f"Average cycle size: {structure_df['file_size_mb'].mean():.1f} MB")
    print(f"Size range: {structure_df['file_size_mb'].min():.1f} - {structure_df['file_size_mb'].max():.1f} MB")

=== DATA STRUCTURE SUMMARY ===
Total cycles: 587
Unique patients: 20
Cycles per patient: 29.4

=== PHASE COUNT DISTRIBUTION ===
10 phases: 587 cycles (100.0%)

=== VOLUME SHAPES ===
(50, 512, 512): 505 cycles (86.0%)
(118, 512, 512): 3 cycles (0.5%)
(113, 512, 512): 3 cycles (0.5%)
(115, 512, 512): 3 cycles (0.5%)
(125, 512, 512): 3 cycles (0.5%)
(117, 512, 512): 3 cycles (0.5%)
(104, 512, 512): 3 cycles (0.5%)
(110, 512, 512): 3 cycles (0.5%)
(114, 512, 512): 3 cycles (0.5%)
(103, 512, 512): 3 cycles (0.5%)
(107, 512, 512): 2 cycles (0.3%)
(133, 512, 512): 2 cycles (0.3%)
(94, 512, 512): 2 cycles (0.3%)
(137, 512, 512): 2 cycles (0.3%)
(132, 512, 512): 2 cycles (0.3%)
(99, 512, 512): 2 cycles (0.3%)
(88, 512, 512): 2 cycles (0.3%)
(122, 512, 512): 2 cycles (0.3%)
(142, 512, 512): 2 cycles (0.3%)
(91, 512, 512): 2 cycles (0.3%)
(84, 512, 512): 2 cycles (0.3%)
(105, 512, 512): 2 cycles (0.3%)
(131, 512, 512): 1 cycles (0.2%)
(168, 512, 512): 1 cycles (0.2%)
(158, 512, 512): 1 cycles (0.

## 2. Motion Analysis Between Phases

In [4]:
def analyze_single_cycle_motion(patient_id, cycle_id):
    """Analyze motion patterns within a single breathing cycle"""
    
    cycle_path = CYCLES_ROOT / patient_id / cycle_id
    
    if not cycle_path.exists():
        return None
    
    # Load all phases
    phase_files = sorted(cycle_path.glob("phase_*.npy"))
    
    if len(phase_files) != 10:
        print(f"Warning: {patient_id}/{cycle_id} has {len(phase_files)} phases, expected 10")
        return None
    
    phases = []
    for phase_file in phase_files:
        try:
            volume = np.load(phase_file)
            phases.append(volume)
        except Exception as e:
            print(f"Error loading {phase_file}: {e}")
            return None
    
    # Calculate motion metrics
    motion_analysis = {
        'patient_id': patient_id,
        'cycle_id': cycle_id,
        'num_phases': len(phases),
        'volume_shape': phases[0].shape
    }
    
    # Motion between consecutive phases
    consecutive_motion = []
    
    for i in range(len(phases)):
        # Next phase (wrap around for last phase)
        next_i = (i + 1) % len(phases)
        
        # Use middle slice for motion analysis
        mid_slice = phases[i].shape[0] // 2
        
        slice1 = phases[i][mid_slice].astype(np.float32)
        slice2 = phases[next_i][mid_slice].astype(np.float32)
        
        # Normalize
        slice1_norm = (slice1 - slice1.mean()) / (slice1.std() + 1e-8)
        slice2_norm = (slice2 - slice2.mean()) / (slice2.std() + 1e-8)
        
        # Calculate motion metrics
        mse = mean_squared_error(slice1_norm.flatten(), slice2_norm.flatten())
        mae = np.mean(np.abs(slice1_norm - slice2_norm))
        
        # Correlation
        correlation = np.corrcoef(slice1_norm.flatten(), slice2_norm.flatten())[0, 1]
        
        consecutive_motion.append({
            'phase_from': i,
            'phase_to': next_i,
            'mse': mse,
            'mae': mae,
            'correlation': correlation
        })
    
    # Overall motion statistics
    motion_analysis['consecutive_motion'] = consecutive_motion
    motion_analysis['avg_mse'] = np.mean([m['mse'] for m in consecutive_motion])
    motion_analysis['max_mse'] = np.max([m['mse'] for m in consecutive_motion])
    motion_analysis['avg_mae'] = np.mean([m['mae'] for m in consecutive_motion])
    motion_analysis['avg_correlation'] = np.mean([m['correlation'] for m in consecutive_motion])
    
    # Find phase with maximum motion
    max_motion_idx = np.argmax([m['mse'] for m in consecutive_motion])
    motion_analysis['max_motion_phase'] = max_motion_idx
    
    return motion_analysis

# Test on a few cycles first
print("Testing motion analysis on sample cycles...")

# Get first few cycles for testing
sample_cycles = structure_df.head(3) if structure_df is not None else []

sample_results = []
for _, row in sample_cycles.iterrows():
    result = analyze_single_cycle_motion(row['patient_id'], row['cycle_id'])
    if result:
        sample_results.append(result)
        print(f"✓ {row['patient_id']}/{row['cycle_id']}: avg_mse={result['avg_mse']:.4f}")

print(f"\nSample motion analysis completed: {len(sample_results)} cycles")

Testing motion analysis on sample cycles...
✓ 116_HM10395/116_HM10395_S106: avg_mse=0.0629
✓ 116_HM10395/116_HM10395_S107: avg_mse=0.0541
✓ 116_HM10395/116_HM10395_S301: avg_mse=0.0070

Sample motion analysis completed: 3 cycles


## 3. Full Motion Analysis (Run Only if Sample Works)

In [ ]:
# Even more optimized version with batch processing and memory efficiency
import numpy as np
from multiprocessing import Pool, cpu_count
import gc
import time

def analyze_batch_motion(batch_args):
    """Process a batch of cycles to reduce overhead"""
    batch_results = []
    
    for patient_id, cycle_id in batch_args:
        try:
            result = analyze_single_cycle_motion(patient_id, cycle_id)
            if result:
                batch_results.append(result)
        except Exception as e:
            print(f"Error processing {patient_id}/{cycle_id}: {e}")
            continue
    
    return batch_results

def create_batches(items, batch_size):
    """Split items into batches"""
    for i in range(0, len(items), batch_size):
        yield items[i:i + batch_size]

# Ultra-optimized version
if len(sample_results) > 0:
    print("Sample analysis successful. Running optimized motion analysis...")
    
    # Prepare arguments
    cycle_args = [(row['patient_id'], row['cycle_id']) for _, row in structure_df.iterrows()]
    
    # Batch processing parameters
    n_processes = max(1, int(cpu_count() * 0.75))
    batch_size = max(1, len(cycle_args) // (n_processes * 4))  # 4 batches per process
    
    print(f"Using {n_processes} processes with batch size {batch_size}")
    print(f"Total batches: {len(list(create_batches(cycle_args, batch_size)))}")
    
    start_time = time.time()
    
    # Create batches
    batches = list(create_batches(cycle_args, batch_size))
    
    # Process batches in parallel
    all_motion_results = []
    
    with Pool(processes=n_processes) as pool:
        # Process batches
        batch_results = list(tqdm(
            pool.imap(analyze_batch_motion, batches),
            total=len(batches),
            desc="Processing batches"
        ))
        
        # Flatten results
        for batch_result in batch_results:
            all_motion_results.extend(batch_result)
    
    elapsed_time = time.time() - start_time
    
    print(f"\nOptimized motion analysis completed: {len(all_motion_results)} cycles")
    print(f"Processing time: {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")
    print(f"Average time per cycle: {elapsed_time/len(all_motion_results):.2f} seconds")
    print(f"Speedup: ~{n_processes:.1f}x faster than sequential")
    
    # Memory cleanup
    gc.collect()
    
    # Save results (same as before)
    motion_summary = []
    for result in all_motion_results:
        motion_summary.append({
            'patient_id': result['patient_id'],
            'cycle_id': result['cycle_id'],
            'num_phases': result['num_phases'],
            'volume_shape': str(result['volume_shape']),
            'avg_mse': result['avg_mse'],
            'max_mse': result['max_mse'],
            'avg_mae': result['avg_mae'],
            'avg_correlation': result['avg_correlation'],
            'max_motion_phase': result['max_motion_phase']
        })
    
    motion_df = pd.DataFrame(motion_summary)
    motion_df.to_csv(OUTPUT_DIR / "motion_analysis.csv", index=False)
    
else:
    print("Sample analysis failed. Skipping full analysis.")
    motion_df = None

Sample analysis successful. Running optimized motion analysis...
Using 4 processes with batch size 36
Total batches: 17


Processing batches:   0%|          | 0/17 [00:00<?, ?it/s]

## 4. Motion Statistics Summary

In [6]:
if motion_df is not None:
    print("=== MOTION ANALYSIS SUMMARY ===")
    print(f"Cycles analyzed: {len(motion_df)}")
    
    print("\n=== MOTION STATISTICS ===")
    print(f"Average MSE: {motion_df['avg_mse'].mean():.6f} ± {motion_df['avg_mse'].std():.6f}")
    print(f"Average MAE: {motion_df['avg_mae'].mean():.6f} ± {motion_df['avg_mae'].std():.6f}")
    print(f"Average Correlation: {motion_df['avg_correlation'].mean():.4f} ± {motion_df['avg_correlation'].std():.4f}")
    
    print("\n=== MOTION MAGNITUDE DISTRIBUTION ===")
    print(f"Low motion (MSE < 0.01): {(motion_df['avg_mse'] < 0.01).sum()} cycles")
    print(f"Medium motion (0.01 ≤ MSE < 0.1): {((motion_df['avg_mse'] >= 0.01) & (motion_df['avg_mse'] < 0.1)).sum()} cycles")
    print(f"High motion (MSE ≥ 0.1): {(motion_df['avg_mse'] >= 0.1).sum()} cycles")
    
    print("\n=== PEAK MOTION PHASES ===")
    phase_counts = motion_df['max_motion_phase'].value_counts().sort_index()
    for phase, count in phase_counts.items():
        print(f"Phase {phase}: {count} cycles ({count/len(motion_df)*100:.1f}%)")
    
    print("\n=== CORRELATION ANALYSIS ===")
    print(f"High correlation (>0.9): {(motion_df['avg_correlation'] > 0.9).sum()} cycles")
    print(f"Medium correlation (0.7-0.9): {((motion_df['avg_correlation'] >= 0.7) & (motion_df['avg_correlation'] <= 0.9)).sum()} cycles")
    print(f"Low correlation (<0.7): {(motion_df['avg_correlation'] < 0.7).sum()} cycles")
else:
    print("No motion analysis data available")

NameError: name 'motion_df' is not defined

## 5. Data Quality Assessment

In [ ]:
def assess_data_quality():
    """Assess data quality for interpolation"""
    
    quality_report = {
        'total_cycles': len(structure_df) if structure_df is not None else 0,
        'valid_cycles': 0,
        'issues': []
    }
    
    if structure_df is None:
        quality_report['issues'].append("No structure data available")
        return quality_report
    
    # Check for standard 10-phase cycles
    standard_cycles = structure_df[structure_df['num_phases'] == 10]
    quality_report['standard_10_phase_cycles'] = len(standard_cycles)
    
    if len(standard_cycles) != len(structure_df):
        non_standard = structure_df[structure_df['num_phases'] != 10]
        quality_report['issues'].append(f"{len(non_standard)} cycles don't have 10 phases")
    
    # Check for consistent shapes
    shape_counts = structure_df['volume_shape'].value_counts()
    most_common_shape = shape_counts.index[0]
    standard_shape_cycles = structure_df[structure_df['volume_shape'] == most_common_shape]
    quality_report['standard_shape_cycles'] = len(standard_shape_cycles)
    quality_report['most_common_shape'] = most_common_shape
    
    if len(shape_counts) > 1:
        quality_report['issues'].append(f"{len(shape_counts)} different volume shapes found")
    
    # Motion quality assessment
    if motion_df is not None:
        # Good motion: not too high (noisy) or too low (static)
        good_motion_cycles = motion_df[
            (motion_df['avg_mse'] > 0.001) & 
            (motion_df['avg_mse'] < 0.5) & 
            (motion_df['avg_correlation'] > 0.5)
        ]
        quality_report['good_motion_cycles'] = len(good_motion_cycles)
        
        # Very low motion (might be static)
        static_cycles = motion_df[motion_df['avg_mse'] < 0.001]
        if len(static_cycles) > 0:
            quality_report['issues'].append(f"{len(static_cycles)} cycles have very low motion (possibly static)")
        
        # Very high motion (might be noisy)
        noisy_cycles = motion_df[motion_df['avg_mse'] > 0.5]
        if len(noisy_cycles) > 0:
            quality_report['issues'].append(f"{len(noisy_cycles)} cycles have very high motion (possibly noisy)")
    
    # Final recommendation
    if motion_df is not None:
        usable_cycles = motion_df[
            (motion_df['num_phases'] == 10) &
            (motion_df['avg_mse'] > 0.001) &
            (motion_df['avg_mse'] < 0.5) &
            (motion_df['avg_correlation'] > 0.5)
        ]
        quality_report['recommended_for_interpolation'] = len(usable_cycles)
    
    return quality_report

# Run quality assessment
quality_report = assess_data_quality()

print("=== DATA QUALITY ASSESSMENT ===")
print(f"Total cycles found: {quality_report['total_cycles']}")
print(f"Standard 10-phase cycles: {quality_report.get('standard_10_phase_cycles', 'N/A')}")
print(f"Standard shape cycles: {quality_report.get('standard_shape_cycles', 'N/A')}")
print(f"Most common shape: {quality_report.get('most_common_shape', 'N/A')}")
print(f"Good motion cycles: {quality_report.get('good_motion_cycles', 'N/A')}")
print(f"Recommended for interpolation: {quality_report.get('recommended_for_interpolation', 'N/A')}")

if quality_report['issues']:
    print("\n=== ISSUES FOUND ===")
    for issue in quality_report['issues']:
        print(f"⚠️  {issue}")
else:
    print("\n✓ No major issues found")

## 6. Save Analysis Results

In [ ]:
# Save all analysis results
print("=== SAVING ANALYSIS RESULTS ===")

# Save structure analysis
if structure_df is not None:
    structure_df.to_csv(OUTPUT_DIR / "structure_analysis.csv", index=False)
    print(f"✓ Structure analysis saved to {OUTPUT_DIR / 'structure_analysis.csv'}")

# Save motion analysis (already saved above)
if motion_df is not None:
    print(f"✓ Motion analysis saved to {OUTPUT_DIR / 'motion_analysis.csv'}")

# Save quality report
with open(OUTPUT_DIR / "quality_report.json", 'w') as f:
    json.dump(quality_report, f, indent=2, default=str)
print(f"✓ Quality report saved to {OUTPUT_DIR / 'quality_report.json'}")

# Create summary for implementation
implementation_summary = {
    'data_ready': quality_report.get('recommended_for_interpolation', 0) > 0,
    'total_cycles': quality_report['total_cycles'],
    'usable_cycles': quality_report.get('recommended_for_interpolation', 0),
    'standard_shape': quality_report.get('most_common_shape', 'Unknown'),
    'major_issues': quality_report['issues'],
    'next_steps': []
}

if implementation_summary['data_ready']:
    implementation_summary['next_steps'].append("Data quality is good for interpolation")
    implementation_summary['next_steps'].append("Can proceed with dataset creation")
    implementation_summary['next_steps'].append("Recommend starting with flow matching approach")
else:
    implementation_summary['next_steps'].append("Data quality issues need to be addressed")
    implementation_summary['next_steps'].append("Review and clean data before interpolation")

with open(OUTPUT_DIR / "implementation_summary.json", 'w') as f:
    json.dump(implementation_summary, f, indent=2, default=str)
print(f"✓ Implementation summary saved to {OUTPUT_DIR / 'implementation_summary.json'}")

print("\n=== ANALYSIS COMPLETE ===")
print(f"All results saved to: {OUTPUT_DIR}")
print(f"Ready for interpolation: {implementation_summary['data_ready']}")
print(f"Usable cycles: {implementation_summary['usable_cycles']}")

In [ ]:
print("\n" + "="*60)
print("CYCLE ANALYZER SUMMARY")
print("="*60)

if structure_df is not None:
    print("📊 DATASET OVERVIEW:")
    print(f"   • Total cycles: {len(structure_df)}")
    print(f"   • Unique patients: {structure_df['patient_id'].nunique()}")
    print(f"   • Cycles per patient: {len(structure_df) / structure_df['patient_id'].nunique():.1f}")
    
    print("\n📐 DATA STRUCTURE:")
    print(f"   • Standard 10-phase cycles: {quality_report.get('standard_10_phase_cycles', 'N/A')}")
    print(f"   • Most common shape: {quality_report.get('most_common_shape', 'N/A')}")
    print(f"   • Total data size: {structure_df['file_size_mb'].sum():.1f} MB")

if motion_df is not None:
    print("\n🔄 MOTION ANALYSIS:")
    print(f"   • Average MSE: {motion_df['avg_mse'].mean():.6f}")
    print(f"   • Average correlation: {motion_df['avg_correlation'].mean():.4f}")
    print(f"   • Good motion cycles: {quality_report.get('good_motion_cycles', 'N/A')}")

print("\n✅ QUALITY ASSESSMENT:")
print(f"   • Recommended for interpolation: {quality_report.get('recommended_for_interpolation', 'N/A')}")
print(f"   • Data ready: {implementation_summary['data_ready']}")

if quality_report.get('issues'):
    print("\n⚠️  ISSUES TO ADDRESS:")
    for issue in quality_report['issues']:
        print(f"   • {issue}")

print("\n🚀 NEXT STEPS:")
for step in implementation_summary['next_steps']:
    print(f"   • {step}")
